### In what proportion of the brain can we confidently say that there are multiple populations of crossing fibers?

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline
import seaborn as sns
sns.set()

In [2]:
import nibabel as nib
import dipy.data as dpd
import dipy.core.gradients as dpg
import dipy.reconst.cross_validation as xval

In [3]:
import urllib as url
import os.path as op

top_url = 'https://stacks.stanford.edu/file/druid:ng782rw8378/'
for b in [1000, 2000, 4000]:
    for ext in ['.nii.gz', '.bvecs', '.bvals']:
        fname = 'SUB1_b%s_1%s'%(b, ext)
        if not op.exists('./data/' + fname):
            url.urlretrieve(op.join(top_url, fname), './data/' + fname)

In [4]:
hardi_img, gtab, labels_img = dpd.read_stanford_labels()
labels = labels_img.get_data()
t1 = dpd.read_stanford_t1()
t1_data = t1.get_data()

Dataset is already in place. If you want to fetch it again please first remove the folder /Users/arokem/.dipy/stanford_hardi 
All files already in /Users/arokem/.dipy/stanford_hardi.
All files already in /Users/arokem/.dipy/stanford_hardi.


In [5]:
white_matter = (labels == 1) | (labels == 2)

In [6]:
import dipy.reconst.csdeconv as csd
import dipy.reconst.dti as dti
import dipy.reconst.dki as dki

In [7]:
data = {}
gtab = {}
for b in [1000, 2000, 4000]:
    data[b] = nib.load(op.join('data', 'SUB1_b%s_1.nii.gz'%b)).get_data()
    gtab[b] = dpg.gradient_table(op.join('./data', 'SUB1_b%s_1.bvals'%b),
                                 op.join('./data', 'SUB1_b%s_1.bvecs'%b))

### Compare models

We set up a function to fit the models using k-fold cross-validation, and calculate goodness-of-fit as the coefficient of determination.

In [ ]:
def compare_models(gtab, data, mask, folds=2):
    # CSD model setup (requires calculation of a response function. We'll do that automatically first)
    response, ratio = csd.auto_response(gtab, data, roi_radius=10, fa_thr=0.7)
    csd_model = csd.ConstrainedSphericalDeconvModel(gtab, response)
    # DTI model setup:
    dti_model = dti.TensorModel(gtab)
    # Predict with k-fold CV:
    csd_pred = xval.kfold_xval(csd_model, data, folds, response, mask=mask)
    dti_pred = xval.kfold_xval(dti_model, data, folds, mask=mask)
    # Calculate coefficient of determination
    cod_csd = xval.coeff_of_determination(csd_pred, data)
    cod_dti = xval.coeff_of_determination(dti_pred, data)
    return cod_csd, cod_dti

In [ ]:
dti_results = {}
csd_results = {}
for bval in [1000, 2000, 4000]:
    this_data = data[bval]
    this_gtab = gtab[bval]
    cod_csd, cod_dti = compare_models(this_gtab, this_data, white_matter)
    dti_results[bval] = cod_dti
    csd_results[bval] = cod_csd

### Visualize: 

We can visualize this comparison several different ways. The first is to compare the distributions of goodness of fit over the entire white matter

In [ ]:
for bval in [1000, 2000, 4000]:
    this_csd = csd[bval]
    this_dti = dti[bval]
    idx = (csd > -50) * (cod_dti > -50)
    fig, ax = plt.subplots()
    ax.hist(this_csd[idx], color='g', histtype='step', linewidth=5, bins=100, label='CSD')
    ax.hist(this_dti[idx], color='b', histtype='step', linewidth=5, bins=100, label='DTI')
    ax.set_xlabel('$R^2$')
    ax.set_ylabel('# Voxels')
    plt.legend(loc='upper left')
    ax.title('b = %s $s/mm^2$'%bval)